# Introduction to Data Science 
# Lecture 13: Classification 1
*COMP 5360 / MATH 4100, University of Utah, http://datasciencecourse.net/*

In this lecutre, we'll discuss:
* a general overview of classification 
* logistic regression 
* k nearest neighbors (k-NN) 
* generalizability and cross validation 

Recommended Reading: 
* G. James, D. Witten, T. Hastie, and R. Tibshirani, An Introduction to Statistical Learning, Ch. 4 [digitial version available here](http://www-bcf.usc.edu/~gareth/ISL/)
* J. Grus, Data Science from Scratch, Ch.12,16

In [ ]:
# imports and setup

import numpy as np
import pandas as pd
import statsmodels.formula.api as sm

import matplotlib.pyplot as plt
%matplotlib inline  
plt.rcParams['figure.figsize'] = (10, 6)
plt.style.use('ggplot')

## Classification 

Recall that in **regression**, we try to predict a real-valued (quantitative) variable. Examples:
1. Predict house prices based on attributes
+ predict credit score rating based on income, balance, gender, education, etc...


In ** classification**, we try to predict a categorical (qualitative) variable. Examples:
1. Predict whether a bank should issue a person a credit card (yes/no)
+ Predict a hospital patient's diagnosis (stroke, heart attack,...) based on symptoms. 

**Setup:** Given $n$ samples $(x_1,y_1), (x_2,y_2),\ldots,(x_n,y_n)$, where $x_i$ are attributes or features and $y_i$ are categorical variables that you want to predict. 

**Goal:** Develop a rule for predicting the categorical variable $y$ based on the features $x$. 

**Example:** Can the number of hours a student studies predict whether they will pass the exam?

**Example:** Can we predict whether or not a student will be admitted to a graduate program based on their undergraduate performance (GPA, GRE score, prestige of student's undergraduate university)? 

**Example:** The post office uses classification to automatically sort mail by zip code. The digits of the zip code are assigned to one of the *classes:* $0,1,2,\ldots,9$. 

We'll cover several classification methods:

* logistic regression
+ k nearest neighbors
+ trees and random forests
+ support vector machines
* Neural Networks and Deep Learning

**Evaluation:** How can we evaluate how well a classification algorithm has done? One way is via the confusion matrix. The *confusion matrix* is just a table where each column of the matrix represents the number of samples in each predicted class and each row represents the number of samples in each actual class. 

Consider the results from a classifier that is trying to classify 27 images of cats, dogs, and rabbits.
![confusion matrix](./confusionMatrix.png)

This classifier is very good at distinguishing between cats and rabbits but lousy at recognizing dogs...half are misclassified!


## Logistic regression 

In logistic regression, the categorical value, $Y$, that we predict only takes two values. We'll call them 0 and 1. 

**Main idea:** Given the predictor variable  X, we model the *probability* that $Y = 1$ by 
$$
p(X) = p(Y = 1 | \ X) = \frac{e^{\beta_0 + \beta_1 X}}{1+ e^{\beta_0 + \beta_1 X}}
$$

Defining the *logistic function* or *sigmoid function*,  
$$
\textrm{logistic}(x) := \frac{e^x}{1 + e^x} = \frac{1}{1+ e^{-x}},
$$
we can write the above expression more succinctly as 
$$
p(X) = \textrm{logistic}( \beta_0 + \beta_1 X ).  
$$

One can check that the model is equivlant to 
$$
\log \left( \frac{p(X)}{1-p(X)} \right) = \beta_0 + \beta_1 X. 
$$

Sometimes, the function on the left hand side of the above equation is called the *logit function*, 
$$
\textrm{logit}(p) := \log \left( \frac{p}{1-p} \right). 
$$
We can equivalently write the model as 

$$
\textrm{logit} ( p(X) ) = \beta_0 + \beta_1 X.  
$$
It follows that the logistic and logit functions are inverses of each other. 

We interpret $\frac{p(X)}{1-p(X)}$ as being the *odds* that $Y=1$. Note that $p(X)$ is a number between 0 and 1 so that the odds that $Y=1$ is a number between 0 and $\infty$. If $p(X) = 0.5$, then odds = 1.  We also have that $\textrm{logit} ( p(X) )$ is a number between $-\infty$ and $\infty$, so it makes sense that it could be linear in the data (as modeled). 


In [ ]:
logistic = lambda x: 1 / ( 1 + np.exp(-x) )

xs = np.linspace(-10,10,1000)
ys = logistic( xs )
plt.plot( xs, ys, color='black',label='logistic(x)' )

plt.legend(loc='upper left')
plt.xlabel('x')
plt.ylim(-.1,1.1)
plt.show()

##################

odds  = lambda p: p/(1-p)
logit = lambda p: np.log( odds(p) )

ps = np.linspace( 1e-5, 1-1e-5, 1000 )

plt.plot( ps, odds(ps), color='black', label='odds(ps)' )
plt.plot( ps, logit(ps), color='red', label='logit(ps)' )

plt.legend(loc='upper left')
plt.xlabel('p')
plt.ylim(-10,10)

**Logistic regression:** Given samples $(x_i,y_i)$ for $i=1,\ldots,n$, find the *best* values of $\beta_0$ and $\beta_1$ so that 
$$
y = \textrm{logistic}( \beta_0 + \beta_1 X ) 
\qquad \textrm{or} \qquad
\textrm{logit} ( y ) = \beta_0 + \beta_1 X.  
$$
Recall that in linear regression, we found the the coefficients $\beta_0$ and $\beta_1$ by setting up the sum of the squared residuals $\sum_{i} (y_i - \beta_0 - \beta_1 x_i)^2$, setting the partial derivatives with respect to  $\beta_0$ and $\beta_1$ to zero, and solving for  $\hat \beta_0$ and $\hat \beta_1$. 
We can try to do the same thing here, but unlike linear regression, there is no longer a closed form solution for the best coefficients $\hat \beta_0$ and $\hat \beta_1$. We have to use a computer to find them. 

**Statistical inference viewpoint:** 
Above, logistic regression was described from the explanatory viewpoint of finding the best parameters to describe the data. 
Just like as in regression, there is also a statistical inference viewpoint. The categorical variable is modeled as 
$$
y = \begin{cases}
1 &  \beta_0 + \beta_1 x + \epsilon > 0 \\
0 & \text{otherwise}
\end{cases}
$$
where $\epsilon$ is a random variable that is distributed according to the logistic distribution. Here, we seek to infer the values of $\beta_0$ and $\beta_1$ from noisy samples of this variable. 

## Logistic Regression Example:  How many hours do you need to study to pass the exam? 

This example was taken from [wikipedia](https://en.wikipedia.org/wiki/Logistic_regression). 

In [ ]:
hours = [0.50, 0.75, 1.00, 1.25, 1.50, 1.75, 1.75, 2.00, 2.25, 2.50, 2.75, 3.00, 3.25, 3.50, 4.00, 4.25, 4.50, 4.75, 5.00, 5.50] 
pass_Exam = [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1]
df = pd.DataFrame( {"hours": hours, "pass_exam": pass_Exam} )
df

In [ ]:
df.plot.scatter( x='hours', y='pass_exam', s=50 );

For this example, we want to find coefficients $\beta_0$ and $\beta_1$ so that the model
$$
p(X) = \textrm{probability}(\textrm{passing} | \ \textrm{X hours studying} ) = \textrm{logistic}( \beta_0 + \beta_1 X )
$$
best explains the data. 

We can use the 'logit' function from the statsmodels python library. 

In [ ]:
exam_model = sm.logit(formula="pass_exam ~ hours", data=df).fit()
exam_model.summary()

In [ ]:
print( df.columns )

df.plot.scatter( x='hours', y='pass_exam', s=50 )

par = dict( exam_model.params )

xs = np.linspace( df['hours'].min(), df['hours'].max(), 1000 )
ys = logistic( par['Intercept'] + par['hours']*xs )
plt.plot( xs, ys, color='Black' ) # Smushed line

############################
# What does the Least squared regression line look like...
#   - Look at the x-intercept on this (red) line.
#
# plt.plot( xs, par['Intercept'] + par['hours'] * xs, color="red" ) # un-smushed version - regular regression
# plt.ylim( [ -.1, 1.1 ] )

############################
# Where is the 50/50 point?  The location we will use to decide pass / fail? 
#   - (Also see confusion matrix below.)
#
# prob_passing = 0.50
# hours_need_to_pass = logit( prob_passing ) - par['Intercept'] / par['hours'] 

# plt.plot( [hours_need_to_pass, hours_need_to_pass], [-1, 2], 'g' )
# print("need:", hours_need_to_pass)

plt.show()

The plot shows hours vs. whether or not the student passed and the logistic regression curve. The curve predicts the probability that a student will pass, given how many hours they've studied. 

From the plot, we can see that the more hours you spend studying, the more likely you are to pass the exam. *Let this be a lesson!* 

If I study just 2 hours, what is the likelihood that I'll pass? 

In [ ]:
print( "2 hours of studying yields: %.2f%%" % ( logistic(par['Intercept'] + par['hours']*2 ) * 100 ) )

I can also get this from exam_model:

In [ ]:
exam_model.predict( pd.DataFrame( {"hours": [2, 3, 4, 5, 6]}, index=[2, 3, 4, 5, 6] ) )

After studying just 2 hours, I'm only $26\%$ likely to pass. 

How many hours do you have to study in order for the probability of passing to be greater than $50\%$?

From the plot, it looks like $\approx 2.75$ hours. How can we find this number from the logistic regression? 

Recall: 
$$
p(X) = \textrm{logistic}( \beta_0 + \beta_1 X )
$$

Note: The $logit()$ function is the inverse of the $logistic()$ function.  p(X) is the probability of passing in this case if you study X hours.  So solving for X:
$$
logit( prob\_pass ) = \beta_0 + \beta_1 X \\
X = ( logit( prob\_pass ) - \beta_0 ) / \beta_1
$$


In [ ]:
prob_passing = 0.75
print( "Hours needed: %.2f" % ( ( logit(prob_passing) - par['Intercept'] ) /par['hours'] ) )

We can also make a confusion matrix for the model.  The confusion matrix shows us the number of data points we correctly classified (either as passing or failing), and the number of data points our model got wrong...

In [ ]:
df['pred_pass'] = exam_model.predict() > .5
pd.crosstab( index=df["pass_exam"], columns=df["pred_pass"] ) # confusion matrix

We interpret this as: for each category (pass,fail), there were two mistakenly classified points. 

## Logistic Regression Example: Graduate school admission based on undergraduate performance 

This example is based on [this blog post](http://blog.yhat.com/posts/logistic-regression-python-rodeo.html). 


**Dataset:**: For 400 students, we have the students attributes: GRE score, GPA, and the "prestige" of the student's undergraduate university (scale of 1 to 4, 1 best). We also have the variable that we want to predict: will they be accepted to grad school (yes/no)? 


In [ ]:
df = pd.read_csv("admittance.csv")

# rename the 'rank' column because there is also a DataFrame method called 'rank'
df.columns = ["admit", "gre", "gpa", "prestige"]

#there are 400 students with 4 attributes: admit, gre, gpa, prestige
print(df.shape)
df.head()

In [ ]:
df.describe()

In [ ]:
_ = pd.plotting.scatter_matrix( df, figsize=(12, 12), diagonal='hist' )

### A first attempt at logistic regression
Let's first just focus on the GRE score. Our model is 
$$
\textrm{logit} ( \textrm{probability admittance} ) = \beta_0 + \beta_1 * \textrm{GRE}.  
$$

In [ ]:
grad_model = sm.logit( formula="admit ~ gre", data=df ).fit()
grad_model.summary()

In [ ]:
df.plot.scatter( x='gre', y='admit' )

xs = np.linspace( df['gre'].min(), df['gre'].max(), 1000 )
par = dict( grad_model.params )
_ = plt.plot( xs, logistic( par['Intercept'] + par['gre']*xs) , color='Black' )

In [ ]:
df[ 'pred_admit_1' ] = grad_model.predict() > .5 
display( pd.crosstab( index=df["admit"], columns=df[ "pred_admit_1" ] ) )

With only the GRE score, we basically just predict that everyone's probability of being admitted to graduate school $< 50\%$.  This is reflected in the confusion matrix. 

We need to include more data into our model! 

### A second attempt at logistic regression

Include both GPA and GRE scores. Our model is 
$$
\textrm{logit} ( \textrm{probability admittance} ) = \beta_0 + \beta_1 * \textrm{GRE} + \beta_2 * \textrm{GPA} .  
$$

In [ ]:
grad_model = sm.logit( formula="admit ~ gre + gpa", data=df ).fit()
grad_model.summary()

In [ ]:
df[ 'pred_admit_2' ] = grad_model.predict() > .5
print( "Pandas Confusion Matrix:" )
print( pd.crosstab( index=df["admit"], columns=df["pred_admit_2"] ) ) # confusion matrix 

# print( "SkLearn Confusion Matrix:" )
# from sklearn.metrics import confusion_matrix
# cm = confusion_matrix( df.admit, df.pred_admit_2 )
# print(cm)

In [ ]:
gpa_model = sm.logit( data=df, formula="admit ~ gpa" ).fit()
gpa_model.summary()

In [ ]:
df.plot.scatter(x='gpa',y='admit')

x = np.linspace(df['gpa'].min(),df['gpa'].max(),1000)
par = dict(gpa_model.params)
plt.plot(x,logistic(par['Intercept'] + par['gpa']*x),color='Black');

In [ ]:
df['pred_admit_3'] = gpa_model.predict()>.5
pd.crosstab(index=df["admit"], columns=df["pred_admit_3"]) # confusion matrix

We're still predicting that most students are not being admitted - still not a very good model. This isn't so surprising if we look at a plot of GPA vs. GRE and try to see which students are admitted. You can't tell with your eye, so it's not surprising that the model fails too. (This is generally a good check.) 

In [ ]:
ax = df[ df['admit'] == 0 ].plot.scatter( x='gpa',y='gre',color='red',label='not admitted' )
df[ df['admit'] == 1 ].plot.scatter(x='gpa',y='gre',color='blue',label='admitted', ax=ax )

xs = np.linspace( 3.5, 4.2, 1000 )
par = dict( grad_model.params )
# solve 0.5 = logistic(b0 + b1*xs + b2*y) for y
# logit(0.5) = b0 + b1*xs + b2*y
# y = (logit(.5) - b0 - b1*x)/b2
plt.plot( xs, (1/par['gre'])*(logit(.5) - par['Intercept'] - par['gpa']*xs), color='Black' )

plt.xlabel('GPA')
plt.ylabel('GRE')
plt.legend(loc='upper left')
plt.show()

### A third attempt

Finally, we include the 'prestige' of the school as a predictor variable.

To get an idea of how many students are admitted from schools of different prestige, we can use the *crosstab* function in the pandas library. 

In [ ]:
pd.crosstab( df['admit'], df['prestige'], rownames=['admit'] )

We see that a smaller percentage of students are admitted from less prestigious universities. 

Recall that in linear regression, you can regress on ordinal variables by creating *dummy* variables (except baseline variable). We did this for the ethnicity variable in the credit dataset. One can similarly consider the prestige variable to be an ordinal variable and create a dummy variables...this is done [here](http://blog.yhat.com/posts/logistic-regression-python-rodeo.html) if you want to read about it. 

I found that the results are not much worse (and simpler) if we consider prestige to be an interval variable (equal increments between prestige levels). 

In [ ]:
grad_model = sm.logit( formula="admit ~ gre + gpa + prestige", data=df ).fit()
grad_model.summary()

In [ ]:
df[ 'pred_admit_3' ] = grad_model.predict() > .5
pd.crosstab( index=df["admit"], columns=df["pred_admit_3"] ) # confusion matrix

We're still now correctly predicting ($253/273=92\%$)  of the students who are not admitted. However, we're only correctly predicting ($29/127 = 23\%$) of the students who are admitted. 

**Moral:** GRE, gpa, and 'prestige' alone do not predict admittance to graduate school. 



In [ ]:
df['adjusted_prestige'] = 5 - df['prestige']

In [ ]:
grad_model = sm.logit( formula="admit ~ gre + gpa + prestige + gpa*adjusted_prestige", data=df ).fit()
grad_model.summary()

In [ ]:
import numpy as np
xs = np.linspace( 1, 10, 30 )
ys = np.abs( xs - 5 ) < 1
plt.scatter( xs, ys )

In [ ]:
bad = pd.DataFrame(data = {"x" : xs, "y" : ys})
bad['y'] = bad['y'].map( {False: 0, True: 1})
bad.head()

In [ ]:
badmodel = sm.logit(data=bad, formula = "y ~ x").fit()
badmodel.summary()

In [ ]:
bad.plot.scatter(x='x',y='y')

x = np.linspace(bad['x'].min(),bad['x'].max(),1000)
par = dict(badmodel.params)
plt.plot(x,logistic(par['Intercept'] + par['x']*x),color='Black');

In [ ]:
xs = np.linspace(1, 10, 20)
ys = np.abs(xs > 5) 
plt.scatter(xs, ys)

In [ ]:
good = pd.DataFrame(data = {"x" : xs, "y" : ys})
good['y'] = good['y'].map( {False: 0, True: 1})
good.head()

In [ ]:
good.describe()

In [ ]:
goodmodel = sm.logit(data=good, formula = "y ~ x").fit()
goodmodel.summary()

In [ ]:
good.plot.scatter(x='x',y='y')

x = np.linspace(good['x'].min(),good['x'].max(),1000)
par = dict(goodmodel.params)
plt.plot(x,logistic(par['Intercept'] + par['x']*x),color='Black');

In [ ]:
from scipy.stats import uniform
xs = uniform.rvs(loc = -2, scale=4, size=1000)
ys = uniform.rvs(loc = -2, scale=4, size=1000)
classLabel = (xs*xs + ys*ys) < 1.5**2

cf = pd.DataFrame({'x': xs, 'y': ys, 'label':classLabel})
cf['label'] = cf['label'].map({False: 0, True: 1})
cf.head()

In [ ]:
ax = cf[cf['label'] == 1].plot.scatter('x', 'y', c='blue')
cf[cf['label']== 0].plot.scatter('x', 'y', c='red', ax=ax)


In [ ]:
circle_model = sm.logit("label ~ x + y", data=cf).fit()
circle_model.summary()

In [ ]:
ax = cf[cf['label'] == 1].plot.scatter('x', 'y', c='blue')
cf[cf['label']== 0].plot.scatter('x', 'y', c='red', ax=ax)

# 0.5 = logistic(int + beta0*x + beta1*y)
# logit(.5) = int + beta0*x + beta1*y
# y = (logit .5 - int - beta0*x)/(beta 1)



x = np.linspace(-2,2,100)
par = dict(circle_model.params)
plt.plot(x,(1/par['y'])*(logit(.5) - par['Intercept'] - par['x']*x),color='Black')

plt.show()

In [ ]:
from scipy.stats import norm
cf['dist'] = cf['x']*cf['x'] + cf['y']*cf['y'] + (norm.rvs(size=len(cf))/4)
cf['dist'].describe()

In [ ]:
distance_model = sm.logit("label ~ dist", data=cf).fit()
distance_model.summary()